In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from torch.utils.data import DataLoader
from helpers import read_hdf5, CombineDataset
from lenshelpers import Residual

from model.helpers_model import NeuralNet

import warnings
warnings.filterwarnings('ignore') 

In [ ]:
# Build the four classes 
config_repo_model = 'data/configFile/config_model'
ratio = 0.5
percent = 0.02

size = 3000
for i in np.arange(1,4):
    #model_name = config_repo_model + str(i) + '.yaml'
    res = Residual()
    res.build(size, i, ratio = ratio, per_error = percent)

print('Data Generation Finished')

In [ ]:
metadata = pd.DataFrame()

for i in np.arange(1,4):
    str_ID = "E"+str(i)+"P"+str(int(percent*100))+"R"+str(int(ratio*100))
    [img, meta] = read_hdf5(str_ID, path = "data/dataSet/")
    metadata = pd.concat([metadata,meta], ignore_index=True)
    if i == 1:
        final_array = img
    else:
         final_array = np.concatenate((final_array, img))
metadata ['ID'] = np.arange(0,final_array.shape[0])
data_set = CombineDataset(metadata,'ID','class',final_array)

print('Reading Data Finished')
data_train, data_test = train_test_split(data_set,train_size=0.9,random_state=42)

batch_size = 50
max_epoch = 70


loader_train = DataLoader(data_train, batch_size = batch_size, shuffle = True, 
                          num_workers = 0, drop_last=True)

loader_test = DataLoader(data_test, batch_size = batch_size, shuffle = True, 
                         num_workers = 0, drop_last=True)

In [ ]:
test_accSGD = np.zeros(max_epoch)
netbasic = NeuralNet('BasicCNN', 'Adam')
for epoch in range(max_epoch):
    netbasic.train(loader_train)
    res = netbasic.test(loader_test,epoch)
    test_accSGD[epoch] = res
    
print('Finished Training')

In [ ]:
test_accADA = np.zeros(max_epoch)
netbasic = NeuralNet('BasicTabular', 'Adam')
for epoch in range(max_epoch):
    netbasic.train(loader_train)
    res = netbasic.test(loader_test,epoch)
    test_accADA[epoch] = res
    
print('Finished Training')

In [ ]:
test_accSGDMOM = np.zeros(max_epoch)
netbasic = NeuralNet('BasicCNNTabular', 'Adam')
for epoch in range(max_epoch):
    netbasic.train(loader_train)
    res = netbasic.test(loader_test,epoch)
    test_accSGDMOM[epoch] = res
    
print('Finished Training')